# Data Collection for transfer analysis


## Imports
We will be using bs4 to parse through the html pages

In [0]:
from bs4 import BeautifulSoup
import urllib.request
import re

import pandas as pd
import numpy as np

from urllib.request import Request, urlopen
from random import randint,shuffle
import time
from time import sleep


In [0]:
!pip install user_agent
import user_agent
from user_agent import generate_user_agent, generate_navigator

## Scraping News Websites for traffic news

### Skysports
Skysports does not hold old transfer news (only 12 pages of it) on their website but they are lenient when it comes to scrappers

In [0]:
content = []
skysports = [
       "https://www.skysports.com/football/transfer-news",
       "https://www.skysports.com/football/transfer-news/more/2",
       "https://www.skysports.com/football/transfer-news/more/3",
       "https://www.skysports.com/football/transfer-news/more/4",
       "https://www.skysports.com/football/transfer-news/more/5",
       "https://www.skysports.com/football/transfer-news/more/6",
       "https://www.skysports.com/football/transfer-news/more/7",
       "https://www.skysports.com/football/transfer-news/more/8",
       "https://www.skysports.com/football/transfer-news/more/9",
       "https://www.skysports.com/football/transfer-news/more/10",
       "https://www.skysports.com/football/transfer-news/more/11",
       "https://www.skysports.com/football/transfer-news/more/12"
       ]
for url in skysports:
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    regex = re.compile('news-item')
    content_lis = soup.find_all('div', attrs={'class': regex})
    for div in content_lis:
        title = div.find('h2').getText()
        link = div.find('a')["href"]
        x = re.search("^/transfer" , link)
        if x != None :
          link = "https://www.skysports.com" + link;
        topic = div.find('strong').getText().strip()
        newspage = urllib.request.urlopen(link)
        tsoup = BeautifulSoup(newspage, 'html.parser')
        try:
          article_title = tsoup.find('h1').getText()
        except:
          article_title = ""
        try:
          article_subtitle = tsoup.find('h2').getText()
        except:
          article_subtitle = ""
        h3_array = tsoup.find_all('h3')
        article_part = []
        for h3 in h3_array : 
            article_part.append(h3.getText())
        article_part = '\n'.join(article_part)
        p_array = tsoup.find_all('p')
        article_content = []
        for h3 in p_array : 
            article_content.append(h3.getText())
        article_content = '\n'.join(article_content)
        regex = re.compile("highlight")
        try:
          date = tsoup.find("span" , attrs={"class":regex}).getText()
        except:
          date = ""
        article = {
            "url" : link ,
            "subscript" : title ,
            "topic" : topic ,
            "title" : article_title ,
            "subtitle" : article_subtitle ,
            "emphasis" : article_part ,
            "content" : article_content ,
            "date" : date
        }
        content.append(article)
df = pd.DataFrame(content)
df.to_csv("skysport_data.csv")

### Goal
Goal.com has all its articles accessible but they have a strict robots.txt so we'll have to use a user agent to scrape data
We'll also put a time delay as we are scraping a lot of information we do not want overload their servers with requests.

* **Irresponsible web scraping can get you blocked from websites** 
* **Data scraped belongs to goal.com. Any form of redistribution will have legal consequences**

In [0]:
headers = {'User-Agent': generate_user_agent()}
content = []
links = []
goal = [
        "https://www.goal.com/en-us/transfer-news"
       ]
for i in range(2,501):
  new_url = goal[0]+"/"+str(i)
  goal.append(new_url)
shuffle(goal)

In [0]:
import time
n = 0
program_starts = time.time()
for url in goal:
  print(url)
  now = time.time()
  print("{0} sites have been scraped .It has been {1} seconds".format(n,now - program_starts))
  sleep(randint(1,30))
  request = Request(url, headers=headers)
  page = urllib.request.urlopen(request)
  soup = BeautifulSoup(page, 'html.parser')
  content_lis = soup.find_all('article')
  for article in content_lis :
    regex = re.compile("article")
    link = article.find('a', attrs={'class': regex})
    if link != None:
      link = "https://www.goal.com"+link["href"]  
      links.append(link)
  n = n + 1

In [0]:
links = pd.DataFrame(links)
links.to_csv("goal.csv")
links

In [0]:
df = pd.read_csv("skysport_data.csv")
links = pd.read_csv("goal.csv")
links = list(links["0"])

In [0]:
shuffle(links)
links

In [0]:
headers = {'User-Agent': generate_user_agent()}
n = 0
program_starts = time.time()
for url in links:
  try:
    n = n+1
    request = Request(url, headers=headers)
    page = urllib.request.urlopen(request)
    soup = BeautifulSoup(page, 'html.parser')
    sleep(randint(1,10))
    try:
      article_title = soup.find("h1").getText()
    except:
      article_title = ""
    try:
      regex = re.compile("teaser")
      article_subtitle = soup.find("div",attrs={"class" : regex}).getText()
    except:
      article_subtitle = ""
    h3_array = soup.find_all('h2')
    article_part = []
    for h3 in h3_array : 
        article_part.append(h3.getText())
    article_part = '\n'.join(article_part)
    p_array = soup.find_all('p')
    article_content = []
    for h3 in p_array : 
        article_content.append(h3.getText())
    article_content = '\n'.join(article_content)
    regex = re.compile("tags-list__link")
    taglist = soup.find_all("a",attrs={"class" : regex})
    tags = []
    for tag in taglist:
      tags.append(tag.getText())
    topic = '+'.join(tags)
    datetag = soup.find("time")
    date = datetag.getText()
    article = {
              "url" : url ,
              "subscript" : article_title ,
              "topic" : topic ,
              "title" : article_title ,
              "subtitle" : article_subtitle ,
              "emphasis" : article_part ,
              "content" : article_content ,
              "date" : date.strip()
          }
    df = df.append(article , ignore_index=True)  
    df.to_csv("transfer_data.csv")
    now = time.time()
    print("{0} sites have been scraped .It has been {1} seconds".format(n,now - program_starts))
  except:
    n=n-1

In [0]:
df.to_csv("final.csv")

In [0]:
df